In [2]:
# -*- coding: utf-8 -*-

import os
import sys
import gc
import joblib
sys.path.append(os.path.join('..', '2. FE'))
import FE_innData as FEin
import FE_extData as FEex

import numpy as np
import pandas as pd
import argparse
from tqdm import tqdm
from datetime import datetime

In [57]:
import imp
imp.reload(FEin)

<module 'FE_innData' from '../2. FE/FE_innData.py'>

In [110]:
df = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_방송편성표추천데이터.xlsx'))

In [111]:
df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-01-01 06:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 07:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 08:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01 09:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-01-01 11:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-01-01 12:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-01-01 13:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-01-01 14:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-01-01 15:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
df['방송날'] = df['방송일시'].dt.date    
df['방송년도'] = df['방송일시'].dt.year
df['방송월'] = df['방송일시'].dt.month
df['방송일'] = df['방송일시'].dt.day
df['방송시간(시간)'] = df['방송일시'].dt.hour
df['방송시간(분)'] = df['방송일시'].dt.minute

In [113]:
df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,방송날,방송년도,방송월,방송일,방송시간(시간),방송시간(분)
0,2020-01-01 06:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,6,20
1,2020-01-01 07:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,7,20
2,2020-01-01 08:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,8,20
3,2020-01-01 09:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,9,20
4,2020-01-01 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,10,20
5,2020-01-01 11:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,11,20
6,2020-01-01 12:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,12,20
7,2020-01-01 13:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,13,20
8,2020-01-01 14:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,14,20
9,2020-01-01 15:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,2020,1,1,15,20


In [36]:
# 날씨 데이터 FE
w_19 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2019_weather.csv'), encoding = 'cp949', dtype='unicode')
w_20 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2020_weather.csv'), encoding = 'cp949', dtype='unicode')
df_wth = pd.concat([w_19, w_20], axis = 0)
df_wth = FEex.preprocessing_weather(df_wth)

# 미세먼지 데이터
dust_2019 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2019_dust.csv'), encoding = 'cp949')
dust_2020 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2020_dust.csv'), encoding = 'cp949')
df_dust = pd.concat([dust_2019, dust_2020], axis = 0)
df_dust = FEex.preprocessing_dust(df_dust, 'train')

In [114]:
df = df.merge(df_wth, left_on = ['방송년도', '방송월', '방송일', '방송시간(시간)'], right_on = ['연도', '월', '일' ,'시간'], how = 'left').drop(['연도', '월', '일', '시간'], axis = 1)
df = df.merge(df_dust, left_on = ['방송년도', '방송월', '방송일', '방송시간(시간)'], right_on = ['연도', '월', '일', '시간'], how ='left').drop(['연도', '월', '일', '시간'], axis = 1)

In [117]:
df_eco = FEex.preprocessing_economy()
# 외부 데이터 merge
df = df.merge(df_eco, left_on = ['방송년도', '방송월'], right_on = ['연도', '월'], how = 'left').drop(['연도', '월'], axis = 1)

In [120]:
df = FEin.engineering_DatePrice(df, 'recommend')

In [122]:
df['방송시간(시간)']

0      6
1      7
2      8
3      9
4     10
5     11
6     12
7     13
8     14
9     15
10    16
11    17
12    18
13    19
14    20
15    21
16    22
17    23
18    24
19     1
Name: 방송시간(시간), dtype: int64

In [104]:
df = df[list(df.columns[:1]) + list(df.columns[8:14]) + list(df.columns[1:8]) + list(df.columns[14:])]

In [108]:
df['방송시간(시간)'] = df['방송일시'].dt.hour

,방송일시,방송날,방송년도,방송월,방송일,방송시간(시간),방송시간(분),노출(분),마더코드,상품코드,...,pca_1,pca_2,pca_3,pca_4,pca_5,isHoliday,평일여부,방송시간대,계절,분기
0,2020-01-01 06:20:00,2020-01-01,2020,1,1,6,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,아침,겨울,1분기
1,2020-01-01 07:20:00,2020-01-01,2020,1,1,7,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,아침,겨울,1분기
2,2020-01-01 08:20:00,2020-01-01,2020,1,1,8,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,아침,겨울,1분기
3,2020-01-01 09:20:00,2020-01-01,2020,1,1,9,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,아침,겨울,1분기
4,2020-01-01 10:20:00,2020-01-01,2020,1,1,10,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,오전,겨울,1분기
5,2020-01-01 11:20:00,2020-01-01,2020,1,1,11,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,오전,겨울,1분기
6,2020-01-01 12:20:00,2020-01-01,2020,1,1,12,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,오전,겨울,1분기
7,2020-01-01 13:20:00,2020-01-01,2020,1,1,13,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,오후,겨울,1분기
8,2020-01-01 14:20:00,2020-01-01,2020,1,1,14,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,오후,겨울,1분기
9,2020-01-01 15:20:00,2020-01-01,2020,1,1,15,20,NaN,NaN,NaN,...,-263543.47186,10271.423409,1246.941238,23024.142933,11316.434841,1,0,오후,겨울,1분기


In [123]:
test = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'test_Rec.pkl'))

In [129]:
import os
import sys
import json
import joblib
import argparse
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder, MinMaxScaler
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, SpatialDropout1D, Activation, concatenate, Conv2D, Conv1D, Lambda, GlobalMaxPooling1D
from keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import ReLU, PReLU, LeakyReLU, ELU
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.metrics import top_k_categorical_accuracy, top_k_categorical_accuracy

import WideNDeep as WnD


# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', required = True)
# args = parser.parse_args()

data = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'train_Rec.pkl'))
locals().update(data)

In [131]:
len(COLUMNS), len(CATEGORICAL_COLUMNS), len(CONTINUOUS_COLUMNS)

(232, 5, 227)

In [145]:
COLUMNS = ['방송월',
                 '방송일',
                 '방송시간(시간)',
                 '경상지수',
                 '불변지수',
                 'pca_1',
                 'pca_2',
                 'pca_3',
                 'pca_4',
                 'pca_5',
                 '강수량(mm)_경기',
                 '강수량(mm)_광주',
                 '강수량(mm)_대구',
                 '강수량(mm)_대전',
                 '강수량(mm)_부산',
                 '강수량(mm)_서울',
                 '강수량(mm)_울산',
                 '강수량(mm)_인천',
                 '기온(°C)_경기',
                 '기온(°C)_광주',
                 '기온(°C)_대구',
                 '기온(°C)_대전',
                 '기온(°C)_부산',
                 '기온(°C)_서울',
                 '기온(°C)_울산',
                 '기온(°C)_인천',
                 '습도(%)_경기',
                 '습도(%)_광주',
                 '습도(%)_대구',
                 '습도(%)_대전',
                 '습도(%)_부산',
                 '습도(%)_서울',
                 '습도(%)_울산',
                 '습도(%)_인천',
                 '시정(10m)_경기',
                 '시정(10m)_광주',
                 '시정(10m)_대구',
                 '시정(10m)_대전',
                 '시정(10m)_부산',
                 '시정(10m)_서울',
                 '시정(10m)_울산',
                 '시정(10m)_인천',
                 '지면온도(°C)_경기',
                 '지면온도(°C)_광주',
                 '지면온도(°C)_대구',
                 '지면온도(°C)_대전',
                 '지면온도(°C)_부산',
                 '지면온도(°C)_서울',
                 '지면온도(°C)_울산',
                 '지면온도(°C)_인천',
                 '체감온도_경기',
                 '체감온도_광주',
                 '체감온도_대구',
                 '체감온도_대전',
                 '체감온도_부산',
                 '체감온도_서울',
                 '체감온도_울산',
                 '체감온도_인천',
                 '풍속(m/s)_경기',
                 '풍속(m/s)_광주',
                 '풍속(m/s)_대구',
                 '풍속(m/s)_대전',
                 '풍속(m/s)_부산',
                 '풍속(m/s)_서울',
                 '풍속(m/s)_울산',
                 '풍속(m/s)_인천',
                 '최고PM10_경기',
                 '최고PM10_광주',
                 '최고PM10_부산',
                 '최고PM10_서울',
                 '최고PM10_울산',
                 '최고PM10_인천',
                 '최고PM25_경기',
                 '최고PM25_광주',
                 '최고PM25_대구',
                 '최고PM25_대전',
                 '최고PM25_부산',
                 '최고PM25_서울',
                 '최고PM25_울산',
                 '최고PM25_인천',
                 '최저PM10_경기',
                 '최저PM10_광주',
                 '최저PM10_대구',
                 '최저PM10_대전',
                 '최저PM10_부산',
                 '최저PM10_서울',
                 '최저PM10_울산',
                 '최저PM10_인천',
                 '최저PM25_경기',
                 '최저PM25_광주',
                 '최저PM25_대구',
                 '최저PM25_대전',
                 '최저PM25_부산',
                 '최저PM25_서울',
                 '최저PM25_울산',
                 '최저PM25_인천',
                 '평균PM10_경기',
                 '평균PM10_광주',
                 '평균PM10_대구',
                 '평균PM10_대전',
                 '최고PM10_대구',
                 '최고PM10_대전',
                 '평균PM10_부산',
                 '평균PM10_서울',
                 '평균PM10_울산',
                 '평균PM10_인천',
                 '평균PM25_경기',
                 '평균PM25_광주',
                 '평균PM25_대구',
                 '평균PM25_대전',
                 '평균PM25_부산',
                 '평균PM25_서울',
                 '평균PM25_울산',
                 '평균PM25_인천',
                 'isHoliday',
                 '평일여부',
                 '방송시간대',
                 '계절',
                 '분기',
                 '일별평균시청률',
                 '일별시간별최대시청률',
                 '일별시간별평균시청률',
                 '일별시간별중간시청률',
                 '시간별월별최대시청률',
                 '시간별월별평균시청률',
                 '시간별월별중간시청률',
                 '월별시간별평균판매량',
                 '월별시간별중간판매량',
                 '월별시간별평균판매단가',
                 '월별시간별중간판매단가',
                 '시간별평균판매량',
                 '시간별중간판매량',
                 '시간별평균판매단가',
                 '시간별중간판매단가',
                 'count_가구',
                 'count_가전',
                 'count_건강기능',
                 'count_농수축',
                 'count_생활용품',
                 'count_속옷',
                 'count_의류',
                 'count_이미용',
                 'count_잡화',
                 'count_주방',
                 'count_침구',
                 'hour_가구',
                 'hour_가전',
                 'hour_건강기능',
                 'hour_농수축',
                 'hour_생활용품',
                 'hour_속옷',
                 'hour_의류',
                 'hour_이미용',
                 'hour_잡화',
                 'hour_주방',
                 'hour_침구',
                 'type1_0',
                 'type1_1',
                 'type1_2',
                 'type1_3',
                 'type1_4',
                 'type1_5',
                 'type1_6',
                 'type1_7',
                 'type1_8',
                 'type1_9',
                 'type1_10',
                 'type1_11',
                 'type1_12',
                 'type1_13',
                 'type1_14',
                 'type1_15',
                 'type1_16',
                 'type1_17',
                 'type1_18',
                 'type1_19',
                 'type1_20',
                 'type2_0',
         'type2_1',
         'type2_2',
         'type2_3',
         'type2_4',
         'type2_5',
         'type2_6',
         'type2_7',
         'type2_8',
         'type2_9',
         'type2_10',
         'type2_11',
         'type2_12',
         'type2_13',
         'type2_14',
         'type2_15',
         'type2_16',
         'type2_17',
         'type2_18',
         'type2_19',
         'type2_20',
         'type2_21',
         'type2_22',
         'type2_23',
         'type2_24',
         'type2_25',
         'type2_26',
         'type2_27',
         'type2_28',
         'type2_29',
         'type2_30',
         'type2_31',
         'type2_32',
         'type2_33',
         'type2_34',
         'type2_35',
         'type2_36',
         'type2_37',
         'type2_38',
         'type2_39',
         'type2_40',
         'type2_41',
         'type2_42',
         'type2_43',
                 'type3_0',
                 'type3_1',
                 'type3_2',
                 'type3_3',
                 'type3_4',
                 'type3_5',
                 'type3_6',
                 'type3_7',
                 'type3_8',
                 'type3_9',
                 'type3_10'
          ]
CATEGORICAL_COLUMNS = ['isHoliday', '평일여부', '방송시간대', '계절', '분기']
CONTINUOUS_COLUMNS = list(set(COLUMNS) - set(CATEGORICAL_COLUMNS))



In [146]:
x_train_continue, x_train_category, x_train_category_poly, train_label = data4train[0], data4train[1], data4train[2], data4train[3]
x_test_continue, x_test_category, x_test_category_poly, test_label = data4valid[0], data4valid[1], data4valid[2], data4valid[3]\

In [147]:

category_inputs, continue_input, deep_model = WnD.Deep_model(X, COLUMNS, CATEGORICAL_COLUMNS, CONTINUOUS_COLUMNS)
wide_model = WnD.Wide_model(x_train_category_poly)



In [148]:
out_layer = concatenate([deep_model, wide_model])
inputs = [continue_input] + category_inputs + [wide_model]
output = Dense(len(train_label[0]), activation='softmax')(out_layer)
model = Model(inputs=inputs, outputs=output)




In [151]:
for i in range(7):
    print(input_data[i].shape)

(30476, 227)
(30476,)
(30476,)
(30476,)
(30476,)
(30476,)
(30476, 16)


In [149]:
early_stopping = EarlyStopping(monitor = 'val_top_3_categorical_accuracy', patience = 30)

input_data = [x_train_continue] + [x_train_category[:, i] for i in range(x_train_category.shape[1])] + [x_train_category_poly]

epochs = 3000
optimizer = Adam(learning_rate=0.01)
batch_size = 64

def top_3_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy',  top_3_categorical_accuracy]
             )

model.fit(input_data,
          train_label,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.2,
          callbacks = [early_stopping]
         )

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24380 samples, validate on 6096 samples
Epoch 1/3000
 1600/24380 [>.............................] - ETA: 11s - loss: nan - accuracy: 0.0881 - top_3_categorical_accuracy: 0.0538  

KeyboardInterrupt: 

In [152]:
COLUMNS = ['방송월',
                 '방송일',
                 '방송시간(시간)',
                 '경상지수',
                 '불변지수',
                 'pca_1',
                 'pca_2',
                 'pca_3',
                 'pca_4',
                 'pca_5',
                 '강수량(mm)_경기',
                 '강수량(mm)_광주',
                 '강수량(mm)_대구',
                 '강수량(mm)_대전',
                 '강수량(mm)_부산',
                 '강수량(mm)_서울',
                 '강수량(mm)_울산',
                 '강수량(mm)_인천',
                 '기온(°C)_경기',
                 '기온(°C)_광주',
                 '기온(°C)_대구',
                 '기온(°C)_대전',
                 '기온(°C)_부산',
                 '기온(°C)_서울',
                 '기온(°C)_울산',
                 '기온(°C)_인천',
                 '습도(%)_경기',
                 '습도(%)_광주',
                 '습도(%)_대구',
                 '습도(%)_대전',
                 '습도(%)_부산',
                 '습도(%)_서울',
                 '습도(%)_울산',
                 '습도(%)_인천',
                 '시정(10m)_경기',
                 '시정(10m)_광주',
                 '시정(10m)_대구',
                 '시정(10m)_대전',
                 '시정(10m)_부산',
                 '시정(10m)_서울',
                 '시정(10m)_울산',
                 '시정(10m)_인천',
                 '지면온도(°C)_경기',
                 '지면온도(°C)_광주',
                 '지면온도(°C)_대구',
                 '지면온도(°C)_대전',
                 '지면온도(°C)_부산',
                 '지면온도(°C)_서울',
                 '지면온도(°C)_울산',
                 '지면온도(°C)_인천',
                 '체감온도_경기',
                 '체감온도_광주',
                 '체감온도_대구',
                 '체감온도_대전',
                 '체감온도_부산',
                 '체감온도_서울',
                 '체감온도_울산',
                 '체감온도_인천',
                 '풍속(m/s)_경기',
                 '풍속(m/s)_광주',
                 '풍속(m/s)_대구',
                 '풍속(m/s)_대전',
                 '풍속(m/s)_부산',
                 '풍속(m/s)_서울',
                 '풍속(m/s)_울산',
                 '풍속(m/s)_인천',
                 '최고PM10_경기',
                 '최고PM10_광주',
                 '최고PM10_부산',
                 '최고PM10_서울',
                 '최고PM10_울산',
                 '최고PM10_인천',
                 '최고PM25_경기',
                 '최고PM25_광주',
                 '최고PM25_대구',
                 '최고PM25_대전',
                 '최고PM25_부산',
                 '최고PM25_서울',
                 '최고PM25_울산',
                 '최고PM25_인천',
                 '최저PM10_경기',
                 '최저PM10_광주',
                 '최저PM10_대구',
                 '최저PM10_대전',
                 '최저PM10_부산',
                 '최저PM10_서울',
                 '최저PM10_울산',
                 '최저PM10_인천',
                 '최저PM25_경기',
                 '최저PM25_광주',
                 '최저PM25_대구',
                 '최저PM25_대전',
                 '최저PM25_부산',
                 '최저PM25_서울',
                 '최저PM25_울산',
                 '최저PM25_인천',
                 '평균PM10_경기',
                 '평균PM10_광주',
                 '평균PM10_대구',
                 '평균PM10_대전',
                 '최고PM10_대구',
                 '최고PM10_대전',
                 '평균PM10_부산',
                 '평균PM10_서울',
                 '평균PM10_울산',
                 '평균PM10_인천',
                 '평균PM25_경기',
                 '평균PM25_광주',
                 '평균PM25_대구',
                 '평균PM25_대전',
                 '평균PM25_부산',
                 '평균PM25_서울',
                 '평균PM25_울산',
                 '평균PM25_인천',
                 'isHoliday',
                 '평일여부',
                 '방송시간대',
                 '계절',
                 '분기',
                 '일별평균시청률',
                 '일별시간별최대시청률',
                 '일별시간별평균시청률',
                 '일별시간별중간시청률',
                 '시간별월별최대시청률',
                 '시간별월별평균시청률',
                 '시간별월별중간시청률',
                 '월별시간별평균판매량',
                 '월별시간별중간판매량',
                 '월별시간별평균판매단가',
                 '월별시간별중간판매단가',
                 '시간별평균판매량',
                 '시간별중간판매량',
                 '시간별평균판매단가',
                 '시간별중간판매단가',
                 'count_가구',
                 'count_가전',
                 'count_건강기능',
                 'count_농수축',
                 'count_생활용품',
                 'count_속옷',
                 'count_의류',
                 'count_이미용',
                 'count_잡화',
                 'count_주방',
                 'count_침구',
                 'hour_가구',
                 'hour_가전',
                 'hour_건강기능',
                 'hour_농수축',
                 'hour_생활용품',
                 'hour_속옷',
                 'hour_의류',
                 'hour_이미용',
                 'hour_잡화',
                 'hour_주방',
                 'hour_침구',
                 'type1_0',
                 'type1_1',
                 'type1_2',
                 'type1_3',
                 'type1_4',
                 'type1_5',
                 'type1_6',
                 'type1_7',
                 'type1_8',
                 'type1_9',
                 'type1_10',
                 'type1_11',
                 'type1_12',
                 'type1_13',
                 'type1_14',
                 'type1_15',
                 'type1_16',
                 'type1_17',
                 'type1_18',
                 'type1_19',
                 'type1_20',
                 'type2_0',
         'type2_1',
         'type2_2',
         'type2_3',
         'type2_4',
         'type2_5',
         'type2_6',
         'type2_7',
         'type2_8',
         'type2_9',
         'type2_10',
         'type2_11',
         'type2_12',
         'type2_13',
         'type2_14',
         'type2_15',
         'type2_16',
         'type2_17',
         'type2_18',
         'type2_19',
         'type2_20',
         'type2_21',
         'type2_22',
         'type2_23',
         'type2_24',
         'type2_25',
         'type2_26',
         'type2_27',
         'type2_28',
         'type2_29',
         'type2_30',
         'type2_31',
         'type2_32',
         'type2_33',
         'type2_34',
         'type2_35',
         'type2_36',
         'type2_37',
         'type2_38',
         'type2_39',
         'type2_40',
         'type2_41',
         'type2_42',
         'type2_43',
                 'type3_0',
                 'type3_1',
                 'type3_2',
                 'type3_3',
                 'type3_4',
                 'type3_5',
                 'type3_6',
                 'type3_7',
                 'type3_8',
                 'type3_9',
                 'type3_10'
          ]

In [153]:
len(COLUMNS)

232